In [ ]:
# Homocedasticidad <- xra q el modelo sea válido tiene q tener homocedasticidad, que no tiene una varianza no constante.
#Si la media de los residuos es cercana a cero, entonces mi modelo está bien.
# Heterocedasticidad
#Hay que equilibrar la muestra y o bien sacar una muestra equilibrada o bien añadir muchos más 1s.

In [3]:
couponPath = "/home/dsc/Data/spark/data/coupon150720.csv"
transmPath = "/home/dsc/Data/spark/data/transm150720.csv"
#tar jxvf data.tar.bz2 para descomprimirlo desde la shell.


In [17]:
ls = sc.textFile(couponPath,3) #3 particiones

ls.getNumPartitions()

3

In [5]:
#Cada una de las líneas del fichero es un cupón.
#Hay 
#numero de transmision
#numero de cupon
#venta que lo vende
#venta que lo asume
#El valor de cada cupon
#clas de booking H
#numero de vuelo
#fecha del vuelo

ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [6]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [7]:
#Como la rdd es resiliente, podemos reusarla
ls.count()

1232662

In [8]:
#La próxima vez que se recalcule se guarda en caché para próximas reutilizaciones.
ls.cache()

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [11]:
ls.count()

1232662

In [12]:
#Spark si puede usar la memoria, usa la memoria, y si no el disco.
#Con persist se puede definir cómo se hacen las caches.
#Con cache siempre a memoria.
#Trozos de la RDD se llaman particiones.
#Hay ciertas operaciones que reparticionas (group by, reduce by) y otra sque no (maps)

ls.persist() #sin argumentos es como caché, particiona a memoria, pero se pueden utilizar argumentos para redefinir
#cómo se guardará la caché.

sc.setSystemProperty # Se puede cambiar la manera en q  cache guarda datos con esta función

<bound method type.setSystemProperty of <class 'pyspark.context.SparkContext'>>

In [ ]:
#Regla básica -> se particiona igual número de partes que cores pero teniendo en cuenta la memoria.
#Balance entre rendimiento y q el cluster funcione. Cada vez que hay una repartición hay tráfico de un nodo a otro.
#Más particiones implican que la red va a estar más saturada de datos.
#Lo ideal es tener pocas particiones pero las suficientes para que no sean demasiado grandes para el nodo,
# de manera que no tengamos demasiado tráfico ni la memoria saturada.

In [50]:
#EJEMPLO# Pasajeros que han volado en Iberia.
lsSplit = ls.map(lambda x: x.split(',') [5])
lsSplit.filter(lambda x: x== "IB").count()

26158

In [25]:
#Composición de funciones:
#Se pueden concatenar las funciones porque map, filter, etc devuelven una rdd

In [52]:
ls.map(lambda x: x.split(',') [5]).filter(lambda x: x== "IB").count()

26158

In [53]:
#Vamos a calcular el importe total vendido por Iberia.
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea,dinero)

In [55]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [58]:
ls.map(getAerolineaDinero).take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [59]:
dineros = ls.map(getAerolineaDinero)

In [62]:
dinerosIB = dineros.filter(lambda x: x[0]=="IB")

In [68]:
dinerosIB.map(lambda x: x[1]/1e6).sum()

2.8280444199999586

In [69]:
cantidades = dinerosIB.map(lambda x: x[1])
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [70]:
#Ahora vamos a obtener qué ruta tiene el billete más caro.

def getRutasDinero (l):
    elems = l.split(",")
    destino = elems[3]
    origen = elems[2]
    ruta = origen + "-" + destino
    dinero = float(elems[6])
    return (ruta,dinero)
    

In [71]:
getRutasDinero(ls.first())

(u'MAA-AUH', 56.79)

In [73]:
rutas = ls.map(getRutasDinero)
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [74]:
rutas.sortBy(lambda x: x[1]).take(5)

[(u'LGA-EWR', 0.0),
 (u'BOM-DEL', 0.0),
 (u'LHR-BOM', 0.0),
 (u'BOM-LHR', 0.0),
 (u'CDG-BRU', 0.0)]

In [75]:
#Para hacer la ordenación descendente, hay que incluir un parámetro False:

rutas.sortBy(lambda x: x[1], False).take(5)

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [77]:
#Vamos a filtrar porque no hay billetes de 6M de dolares. Por poner quitamos 7500 de límite.
rutas.filter(lambda x:x[1]<7500).sortBy(lambda x: x[1],False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [78]:
filtrado = rutas.filter(lambda x:x[1]<7500)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f4b91066c10>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f4b91066e90>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f4bac0a1f50>)]

In [79]:
#En vez de agrupar por cupón agrupamos por ruta.
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f4bac094910>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f4bac094850>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f4bac094ad0>)]

In [88]:
maximosPorRuta = filtrado.groupByKey().mapValues(lambda x: max(x))

In [89]:
maximosPorRuta.sortBy(lambda x: x[1],False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [96]:
mediasPorRuta = filtrado.groupByKey().mapValues(lambda x: sum(x)/len(x))

In [97]:
mediasPorRuta.sortBy(lambda x: x[1],False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [99]:
#ruta, aerolinea, booking class -> vamos a usar estos campos para definir un límite aceptable dentro del precio del cupón.
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [103]:
#Booking class son las dos Hs, usaermos la primera
#las columnas que usaremos son 2,3,6,8

def GetALDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems [3]
    al = elems [4]
    dinero = float(elems [6])
    clase = elems [9]
    return ((ruta,al,clase),dinero)

In [104]:
GetALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [105]:
import math

In [139]:
#Vamos a definir unos valores límite para el importe 

def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1

### nos devuelve el valor límite x grupo

In [140]:
valorLimite([1,5,77,5,44,3,20])

134.12110224788873

In [141]:
rutas = ls.map(GetALDineroClase)

In [142]:
rutas.take(3)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0)]

In [143]:
rutas.groupByKey().take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f4bac03be50>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f4b9101d910>)]

In [144]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [145]:
#Como no nos interesa la clave, sino el valor, usaremos un mapvalues.
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [147]:
grupos.cache()

PythonRDD[206] at RDD at PythonRDD.scala:43

In [149]:
grupos.mapValues(lambda vs: sum(vs)/len(vs)).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 115.0 failed 1 times, most recent failure: Lost task 0.0 in stage 115.0 (TID 214, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1907, in <lambda>
    map_values_fn = lambda kv: (kv[0], f(kv[1]))
  File "<ipython-inp

In [150]:
#COmo la función peta todo el rato, vamos a insertar una lambda más sofisticada.

def mediaGrupo (vs):
    if (len(vs)==0):
        return 0.0
    else:
        return (sum(vs)/len(vs))

In [151]:
grupos.mapValues(mediaGrupo).take(5)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'DPS-TMC', u'GA', u'N'), 60.7925),
 ((u'BEG-CDG', u'JU', u'K'), 0.0),
 ((u'SAV-CLT', u'US', u'N'), 0.0),
 ((u'DUS-AMS', u'KL', u'C'), 0.0)]

In [152]:
#Ahora que ya tenemos la media de cada grupo ya podemos calcular la media.
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1],False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [ ]:
#Los outliers están por aquí, la función de las desviaciones estándar no ha servido.

In [153]:
#Vamos a filtrar aquellas tuplas que no tengan casi valores

grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  [42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   48.67,
   42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76]),
 ((u'DPS-TMC', u'GA', u'N'), [59.15, 59.26, 62.38, 62.38])]

In [154]:
grupos = rutas.groupByKey()

In [155]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f4bac0a19d0>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f4bac0a1cd0>)]

In [158]:
gruposFiltrados = grupos.filter(lambda x: len(x[1]) > 9).mapValues(filtraCuponesExtremos)

In [160]:
gruposFiltrados.take(1)

[((u'TRD-BOO', u'SK', u'V'),
  [42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   48.67,
   42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76])]

In [162]:
r = gruposFiltrados.mapValues(mediaGrupo)
r.take(2)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'PHX-CLT', u'US', u'S'), 157.65285714285713)]

In [163]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

In [164]:
#Ocurrencias en un dataset de valores únicos.
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [168]:
#Ocurrencias en un dataset de valores únicos.
#El collect extrae toda la lista entera del rdd y en este caso solo todo el rdd de esa columna.
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

In [169]:
#Ocurrencias en un dataset de valores únicos.
#El collect extrae toda la lista entera del rdd y en este caso solo todo el rdd de esa columna.
ls.map(lambda x: x.split(",")[7]).distinct().saveAsTextFile("results.csv")